In [1]:
import robosuite 
import dexmimicgen  

import h5py 
import imageio 
import numpy as np
import time 
import os
import json

'''

1. Loading dual robot environment
'''

# def get_env_metadata_from_dataset(dataset_path, ds_format="robomimic"):
#     """
#     Retrieves env metadata from dataset.

#     Args:
#         dataset_path (str): path to dataset

#     Returns:
#         env_meta (dict): environment metadata. Contains 3 keys:

#             :`'env_name'`: name of environment
#             :`'type'`: type of environment, should be a value in EB.EnvType
#             :`'env_kwargs'`: dictionary of keyword arguments to pass to environment constructor
#     """
#     dataset_path = os.path.expanduser(dataset_path)
#     f = h5py.File(dataset_path, "r")
#     if ds_format == "robomimic":
#         env_meta = json.loads(f["data"].attrs["env_args"])
#     else:
#         raise ValueError
#     f.close()
#     return env_meta

from vishal_dev.utils import get_env_metadata_from_dataset

dataset_path = "/home/vishal/Volume_E/Active/Gap_year/grand-project-2025/deepak/dexmimicgen/datasets/two_arm_box_cleanup.hdf5"

env_meta = get_env_metadata_from_dataset(dataset_path)
print(env_meta)

write_to_video = False

env_kwargs = env_meta['env_kwargs']
env_kwargs["env_name"] = env_meta["env_name"]
env_kwargs["has_renderer"] = True
env_kwargs["renderer"] = "mjviewer"
env_kwargs["has_offscreen_renderer"] = write_to_video # False # write_video
env_kwargs["use_camera_obs"] = False

env_kwargs.pop("env_lang")

env = robosuite.make(**env_kwargs)

[robosuite WARNING] No private macro file found! (macros.py:57)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:58)
[robosuite WARNING] To setup, run: python /home/vishal/Volume_E/Active/Gap_year/grand-project-2025/deepak/robosuite/robosuite/scripts/setup_macros.py (macros.py:59)
[robosuite WARNING] Could not import robosuite_models. Some robots may not be available. If you want to use these robots, please install robosuite_models from source (https://github.com/ARISE-Initiative/robosuite_models) or through pip install. (__init__.py:30)
[robosuite WARNING] Could not load the mink-based whole-body IK. Make sure you install related import properly, otherwise you will not be able to use the default IK controller setting for GR1 robot. (__init__.py:40)


{'env_name': 'TwoArmBoxCleanup', 'env_version': '1.5.1', 'type': 1, 'env_kwargs': {'robots': ['PandaDexRH', 'PandaDexLH'], 'controller_configs': {'type': 'BASIC', 'body_parts': {'right': {'type': 'OSC_POSE', 'input_max': 1, 'input_min': -1, 'output_max': [0.05, 0.05, 0.05, 0.5, 0.5, 0.5], 'output_min': [-0.05, -0.05, -0.05, -0.5, -0.5, -0.5], 'kp': 150, 'damping_ratio': 1, 'impedance_mode': 'fixed', 'kp_limits': [0, 300], 'damping_ratio_limits': [0, 10], 'position_limits': None, 'orientation_limits': None, 'uncouple_pos_ori': True, 'input_type': 'delta', 'input_ref_frame': 'base', 'interpolation': None, 'ramp_ratio': 0.2, 'gripper': {'type': 'GRIP', 'use_action_scaling': False}}}}, 'translucent_robot': False, 'env_configuration': 'parallel', 'reward_shaping': False, 'camera_names': ['agentview', 'robot0_eye_in_hand', 'robot1_eye_in_hand'], 'camera_heights': 84, 'camera_widths': 84, 'has_renderer': False, 'has_offscreen_renderer': True, 'ignore_done': True, 'use_object_obs': True, 'use_

In [ ]:
import pose_utils_vishal_from_mimicgen as PoseUtils
import robosuite.utils.transform_utils as T

: 

In [ ]:
'''
Extracting a demonstration from the dataset
'''

data = h5py.File(dataset_path, "r")
print(f"Keys in the dataset: {data['data'].keys()}")
print(f"Number of demonstrations in the dataset: {len(data['data'].keys())}")
demo_id = 'demo_15'
print(f"Length of demo {demo_id}: {data['data'][demo_id]['obs']['robot0_eef_pos'].shape[0]}")
print(f"Observations shape: {data['data'][demo_id]['obs'].keys()}")
print(f"Actions shape: {data['data'][demo_id]['actions'].shape}")
print(f"Actions: {data['data'][demo_id]['actions']}")
print(f"Actions shape: {data['data'][demo_id]['actions'].shape}")
print(f"Action at step 0: {data['data'][demo_id]['actions'][0]}")

action_trajectory = data['data'][demo_id]['actions']
obs_trajectory = data['data'][demo_id]['obs']
states_trajectory = data['data'][demo_id]['states']

eef_poses = data['data'][demo_id]['datagen_info']['eef_pose']
gripper_actions = data['data'][demo_id]['datagen_info']['gripper_action']
target_eef_poses = data['data'][demo_id]['datagen_info']['target_pose']

In [ ]:
from vishal_dev.utils import just_reset, reset_to_state, reset_to

In [ ]:
ep = demo_id
states = data["data/{}/states".format(ep)][()]
initial_state = dict(states=states[0])
initial_state["model"] = data["data/{}".format(ep)].attrs["model_file"]
# if args.use_current_model:
initial_state["model"] = env.sim.model.get_xml()
initial_state["ep_meta"] = data["data/{}".format(ep)].attrs.get("ep_meta", None)

In [ ]:
'''
Replay a demonstration
'''
# reset_to_state(env, states_trajectory[0])
# just_reset(env, states_trajectory[25])

for i in range(action_trajectory.shape[0]):
    action = action_trajectory[i]
    obs, reward, done, info = env.step(action)
    env.render()
    time.sleep(0.05)

In [ ]:
max_dpos = 0.05
max_drot = 0.5

from vishal_dev.utils import eef_to_action_single_arm, eef_to_action

In [ ]:
'''
Data class for source and transformed segments

inputs:
    arm1_src_traj: (N, 4, 4) numpy array
    arm2_src_traj: (N, 4, 4) numpy array
    starting_lid_pose: (4, 4) numpy array
    starting_box_pose: (4, 4) numpy array

to_maintain:
    arm1_src_segment1: (N, 4, 4) numpy array
    arm1_src_segment2: (N, 4, 4) numpy array
    arm2_src_segment1: (N, 4, 4) numpy array
    arm2_src_segment2: (N, 4, 4) numpy array
    starting_lid_pose: (4, 4) numpy array
    starting_box_pose: (4, 4) numpy array
    arm1_tgt_segment1: (N, 4, 4) numpy array
    arm1_tgt_segment2: (N, 4, 4) numpy array
    arm2_tgt_segment1: (N, 4, 4) numpy array
    arm2_tgt_segment2: (N, 4, 4) numpy array
    
    INTERPOLATED TRAJECTORIES
    arm1_interp_toseg1: (N, 4, 4) numpy array
    arm1_interp_toseg2: (N, 4, 4) numpy array
    arm2_interp_toseg1: (N, 4, 4) numpy array
    arm2_interp_toseg2: (N, 4, 4) numpy array
'''

In [ ]:
# obfull[0].keys()

In [ ]:
'''
Now prepare the data
'''
obfull = just_reset(env, states_trajectory[0])
src_lid_pos = data['data'][demo_id]['datagen_info']['object_poses']['lid'][0][:3, 3]
src_lid_mat = data['data'][demo_id]['datagen_info']['object_poses']['lid'][0][:3, :3].reshape(3, 3)
src_box_pos = data['data'][demo_id]['datagen_info']['object_poses']['box'][0][:3, 3]
src_box_mat = data['data'][demo_id]['datagen_info']['object_poses']['box'][0][:3, :3].reshape(3, 3)
src_lid_pose = PoseUtils.make_pose(src_lid_pos, src_lid_mat)
src_box_pose = PoseUtils.make_pose(src_box_pos, src_box_mat)
print("Starting lid pose: ", src_lid_pose)
print("Starting box pose: ", src_box_pose)

new_lid_pos = env.sim.data.body('lid_obj_root').xpos
new_lid_mat = env.sim.data.body('lid_obj_root').xmat.reshape(3, 3)
new_box_pos = env.sim.data.body('box_obj_root').xpos
new_box_mat = env.sim.data.body('box_obj_root').xmat.reshape(3, 3)
new_lid_pose = PoseUtils.make_pose(new_lid_pos, new_lid_mat)
new_box_pose = PoseUtils.make_pose(new_box_pos, new_box_mat)
print("New lid pose: ", new_lid_pose)
print("New box pose: ", new_box_pose)

cur_arm1_eef_pos = obfull[0]['robot0_eef_pos']
cur_arm1_eef_mat = obfull[0]['robot0_eef_quat']
cur_arm1_eef_mat = T.quat2mat(cur_arm1_eef_mat)
cur_arm1_eef_pose = PoseUtils.make_pose(cur_arm1_eef_pos, cur_arm1_eef_mat)
print("Current arm1 eef pose: ", cur_arm1_eef_pose)
cur_arm2_eef_pos = obfull[0]['robot1_eef_pos']
cur_arm2_eef_mat = obfull[0]['robot1_eef_quat']
cur_arm2_eef_mat = T.quat2mat(cur_arm2_eef_mat)
cur_arm2_eef_pose = PoseUtils.make_pose(cur_arm2_eef_pos, cur_arm2_eef_mat)
print("Current arm2 eef pose: ", cur_arm2_eef_pose)
cur_eef_poses = np.zeros((2, 4, 4))
cur_eef_poses[0, :, :] = cur_arm1_eef_pose
cur_eef_poses[1, :, :] = cur_arm2_eef_pose

In [ ]:
'''
Transforming the source trajectories to the new object positions
'''

'''
Transforming two segments one by one

Process:
1. Transform segment 1 using object pose at start of segment 1
2. Interpolate between starting pose to starting pose of segment 1
3. Transform segment 2 using object pose at start of segment 2
4. Interpolate between end of segment 1 to starting pose of segment 2

'''

from pose_utils_vishal_from_mimicgen import transform_source_data_segment_using_object_pose
from pose_utils_vishal_from_mimicgen import interpolate_poses
from pose_utils_vishal_from_mimicgen import make_pose
from pose_utils_vishal_from_mimicgen import unmake_pose

def transform_two_segments_for_one_arm(
        segment1,
        segment2,
        src_obj1_pose,
        cur_obj1_pose,
        src_obj2_pose,
        cur_obj2_pose,
        starting_eef_pose,
    ):
    '''Transform 2 segments of trajectory from source to current based on object poses

    Args:
        segment1 (list): list of eef_poses for segment 1 (T, 4, 4)
        segment2 (list): list of eef_poses for segment 2 (T, 4, 4)
        src_obj1_pose (np.ndarray): source object pose (4, 4)
        cur_obj1_pose (np.ndarray): current object pose (4, 4)
        src_obj2_pose (np.ndarray): source object pose (4, 4)
        cur_obj2_pose (np.ndarray): current object pose (4, 4)
        starting_eef_pose (list): starting eef pose (4, 4)
    Returns:
        
    '''
    # Transform segments into their corresponding target object poses
    transformed_segment1 = transform_source_data_segment_using_object_pose(
        src_eef_poses=segment1,
        src_obj_pose=src_obj1_pose,
        obj_pose=cur_obj1_pose,
    )
    transformed_segment2 = transform_source_data_segment_using_object_pose(
        src_eef_poses=segment2,
        src_obj_pose=src_obj2_pose,
        obj_pose=cur_obj2_pose,
    )

    # Interpolate between starting pose to starting pose of segment 1
    interp_poses0, num_steps = interpolated_poses = interpolate_poses(
        pose_1=starting_eef_pose,
        pose_2=transformed_segment1[0],
        num_steps=10,
        # step_size=0.1, # one of step size or num_steps is needed
    )

    # Interpolate between end of segment 1 to starting pose of segment 2
    interp_poses1, num_steps = interpolated_poses = interpolate_poses(
        pose_1=transformed_segment1[-1],
        pose_2=transformed_segment2[0],
        num_steps=10,
        # step_size=0.1, # one of step size or num_steps is needed
    )

    # Create a dict of all trajectories 
    overall_trajectory = {
        'interp_to_segment1': interp_poses0,
        'segment1': transformed_segment1,
        'interp_to_segment2': interp_poses1,
        'segment2': transformed_segment2,
    }

    return overall_trajectory

def get_transformed_segments_for_both_arms(
        segment1,
        segment2,
        src_obj1_pose,
        cur_obj1_pose,
        src_obj2_pose,
        cur_obj2_pose,
        starting_eef_pose,
    ):
    '''

    args:
        segment1 (np.ndarray): (N, 2, 4, 4)
        segment2 (np.ndarray): (M, 2, 4, 4) 
        src_obj1_pose (np.ndarray): (4, 4)
        cur_obj1_pose (np.ndarray): (4, 4)
        src_obj2_pose (np.ndarray): (4, 4)
        cur_obj2_pose (np.ndarray): (4, 4)
        starting_eef_pose (list): list of two eef poses [(4, 4), (4, 4)]
    returns:
        dict: {
            "arm1": {
                "overall_trajectory": overall_trajectory_arm1 (dict): {
                    'interp_to_segment1': interp_poses0,
                    'segment1': transformed_segment1,
                    'interp_to_segment2': interp_poses1,
                    'segment2': transformed_segment2,
                },
                "starting_eef_pose": starting_eef_pose_arm1 (list): [(4, 4)]
            },
            "arm2": {
                "overall_trajectory": overall_trajectory_arm2 (dict): {
                    'interp_to_segment1': interp_poses0,
                    'segment1': transformed_segment1,
                    'interp_to_segment2': interp_poses1,
                    'segment2': transformed_segment2,
                },
                "starting_eef_pose": starting_eef_pose_arm2 (list): [(4, 4)]
            }
        }
    '''

    arm1_segment1 = segment1[:, 0, :, :]
    arm1_segment2 = segment2[:, 0, :, :]
    arm2_segment1 = segment1[:, 1, :, :]
    arm2_segment2 = segment2[:, 1, :, :]
    starting_eef_pose_arm1 = starting_eef_pose[0] # [starting_eef_pose[0][0], starting_eef_pose[1][0]]
    starting_eef_pose_arm2 = starting_eef_pose[1] # [starting_eef_pose[0][1], starting_eef_pose[1][1]]

    overall_trajectory_arm1 = transform_two_segments_for_one_arm(
        segment1=arm1_segment1,
        segment2=arm1_segment2,
        src_obj1_pose=src_obj1_pose,
        cur_obj1_pose=cur_obj1_pose,
        src_obj2_pose=src_obj2_pose,
        cur_obj2_pose=cur_obj2_pose,
        starting_eef_pose=starting_eef_pose_arm1,
    )

    overall_trajectory_arm2 = transform_two_segments_for_one_arm(
        segment1=arm2_segment1,
        segment2=arm2_segment2,
        src_obj1_pose=src_obj1_pose,
        cur_obj1_pose=cur_obj1_pose,
        src_obj2_pose=src_obj2_pose,
        cur_obj2_pose=cur_obj2_pose,
        starting_eef_pose=starting_eef_pose_arm2,
    )

    return {
        "arm1": {
            "overall_trajectory": overall_trajectory_arm1,
            "starting_eef_pose": starting_eef_pose_arm1
        },
        "arm2": {
            "overall_trajectory": overall_trajectory_arm2,
            "starting_eef_pose": starting_eef_pose_arm2
        }
    }


In [ ]:
cur_eef_poses

In [ ]:
from vishal_dev.utils import segment_trajectory

segments = segment_trajectory(data, demo_id)

src_arm_segment1 = target_eef_poses[:segments[0][-1]+1, :]
src_arm_segment2 = target_eef_poses[:segments[1][-1]+1, :]

full_trajectory_outs = get_transformed_segments_for_both_arms(
    segment1=src_arm_segment1,
    segment2=src_arm_segment2,
    src_obj1_pose=src_lid_pose,
    cur_obj1_pose=new_lid_pose,
    src_obj2_pose=src_box_pose,
    cur_obj2_pose=new_box_pose,
    starting_eef_pose=cur_eef_poses,
)

In [ ]:
# transformed_data = get_transformed_segments_for_both_arms(
#     segment1=eef_poses[0],
#     segment2=eef_poses[1],
#     src_obj1_pose=src_lid_pose,
#     cur_obj1_pose=new_lid_pose,
#     src_obj2_pose=src_box_pose,
#     cur_obj2_pose=new_box_pose,
#     starting_eef_pose=target_eef_poses,
# )

In [ ]:
'''
Execution order/logic for trajectories:

# 1. Interpolate to initial poses for each arm in parallel
N, M = len(arm1_interp_toseg1), len(arm2_interp_toseg1)

for i in range(max(N, M)):
    initialize action with zeros
    if i < N:
        set action[0:12] to arm1_interp_toseg1[i]
    if i < M:
        set action[12:24] to arm2_interp_toseg1[i]
    env.step(action)

# 2. Execute segment1 for each arm in parallel. This segment will be of same length for both arms

for i in range(len(arm1_segment1)):
    initialize action with zeros
    set action[0:12] to arm1_segment1[i]
    set action[12:24] to arm2_segment1[i]
    env.step(action)

# 3. Interpolate to initial poses of segment 2 for each arm in parallel
N, M = len(arm1_interp_toseg2), len(arm2_interp_toseg2)

for i in range(max(N, M)):
    initialize action with zeros
    if i < N:
        set action[0:12] to arm1_interp_toseg2[i]
    if i < M:
        set action[12:24] to arm2_interp_toseg2[i]
    env.step(action)

# 4. Execute segment2 for each arm in parallel. This segment will be of same length for both arms

for i in range(len(arm1_segment2)):
    initialize action with zeros
    set action[0:12] to arm1_segment2[i]
    set action[12:24] to arm2_segment2[i]
    env.step(action)

'''

In [ ]:
'''
Reset environment to new starting poses
'''



In [ ]:
'''
Generate data

'''
interp_arm1_toseg1 = full_trajectory_outs["arm1"]["overall_trajectory"]["interp_to_segment1"]
interp_arm1_toseg2 = full_trajectory_outs["arm1"]["overall_trajectory"]["interp_to_segment2"]
interp_arm2_toseg1 = full_trajectory_outs["arm2"]["overall_trajectory"]["interp_to_segment1"]
interp_arm2_toseg2 = full_trajectory_outs["arm2"]["overall_trajectory"]["interp_to_segment2"]
seg1_arm1 = full_trajectory_outs["arm1"]["overall_trajectory"]["segment1"]
seg1_arm2 = full_trajectory_outs["arm2"]["overall_trajectory"]["segment1"]
seg2_arm1 = full_trajectory_outs["arm1"]["overall_trajectory"]["segment2"]
seg2_arm2 = full_trajectory_outs["arm2"]["overall_trajectory"]["segment2"]

In [ ]:
'''
Execute interpolation to initial poses for each arm in parallel
'''

# obs = obfull[0]
# reset_to(env, initial_state) # Reset this to some other state
# reset_to_state(env, states_trajectory[0])
# obfull = just_reset(env, initial_state) #  --- IGNORE ---

obs = obfull[0].copy()

N, M = len(interp_arm1_toseg1), len(interp_arm2_toseg1)

prev_action = action_trajectory[0]

for i in range(max(N, M)):

    action = prev_action.copy()

    if i == 0:
        if i < N:
            action[0:12] = eef_to_action_single_arm(
                eef_pos=np.stack([PoseUtils.unmake_pose(interp_arm1_toseg1[i])[:3, 3]]),
                eef_rot_mats=np.stack([PoseUtils.unmake_pose(interp_arm1_toseg1[i])[:3, :3]]),
                original_action=interp_arm1_toseg1[i], # action_trajectory[0],
                prev_eef_pos=np.stack([obs['robot0_eef_pos'], obs['robot1_eef_pos']]),
                prev_eef_quat=np.stack([obs['robot0_eef_quat'], obs['robot1_eef_quat']]),
                max_dpos=max_dpos,
                max_drot=max_drot,
                cur_rot_from_env=False, #cTrue
            )
        if i < M:
            action[12:24] = eef_to_action_single_arm(
                eef_pos=np.stack([PoseUtils.unmake_pose(interp_arm2_toseg1[i])[:3, 3]]),
                eef_rot_mats=np.stack([PoseUtils.unmake_pose(interp_arm2_toseg1[i])[:3, :3]]),
                original_action=interp_arm2_toseg1[i], # action_trajectory[0],
                prev_eef_pos=np.stack([obs['robot0_eef_pos'], obs['robot1_eef_pos']]),
                prev_eef_quat=np.stack([obs['robot0_eef_quat'], obs['robot1_eef_quat']]),
                max_dpos=max_dpos,
                max_drot=max_drot,
                cur_rot_from_env=False, #cTrue
            )

    else:   
        if i < N:
            action[0:12] = eef_to_action_single_arm(
                eef_pos=np.stack([PoseUtils.unmake_pose(interp_arm1_toseg1[i])[:3, 3]]),
                eef_rot_mats=np.stack([PoseUtils.unmake_pose(interp_arm1_toseg1[i])[:3, :3]]),
                original_action=interp_arm1_toseg1[i], # action_trajectory[0],
                prev_eef_pos=np.stack([obs['robot0_eef_pos'], obs['robot1_eef_pos']]),
                prev_eef_quat=np.stack([obs['robot0_eef_quat'], obs['robot1_eef_quat']]),
                max_dpos=max_dpos,
                max_drot=max_drot,
                cur_rot_from_env=True
            )   
        if i < M:
            action[12:24] = eef_to_action_single_arm(
                eef_pos=np.stack([PoseUtils.unmake_pose(interp_arm2_toseg1[i])[:3, 3]]),
                eef_rot_mats=np.stack([PoseUtils.unmake_pose(interp_arm2_toseg1[i])[:3, :3]]),
                original_action=interp_arm2_toseg1[i], # action_trajectory[0],
                prev_eef_pos=np.stack([obs['robot0_eef_pos'], obs['robot1_eef_pos']]),
                prev_eef_quat=np.stack([obs['robot0_eef_quat'], obs['robot1_eef_quat']]),
                max_dpos=max_dpos,
                max_drot=max_drot,
                cur_rot_from_env=True
            )

    prev_action = action.copy()

    obs, reward, done, info = env.step(action)
    env.render()
    time.sleep(0.05)


In [ ]:
'''
Execute transformed synced trajectory for each arm in parallel - segment 1

obs carried forward from previous loop
'''

# Assert both segments are of same length
assert len(seg1_arm1) == len(seg1_arm2)

N = len(seg1_arm1)

prev_action = action_trajectory[0]

for i in range(N):
    action = prev_action.copy()

    if i == 0:
        action[0:12] = eef_to_action_single_arm(
            eef_pos=np.stack([PoseUtils.unmake_pose(seg1_arm1[i])[:3, 3]]),
            eef_rot_mats=np.stack([PoseUtils.unmake_pose(seg1_arm1[i])[:3, :3]]),
            original_action=seg1_arm1[i], # action_trajectory[0],
            prev_eef_pos=np.stack([obs['robot0_eef_pos'], obs['robot1_eef_pos']]),
            prev_eef_quat=np.stack([obs['robot0_eef_quat'], obs['robot1_eef_quat']]),
            max_dpos=max_dpos,
            max_drot=max_drot,
            cur_rot_from_env=False, #cTrue
        )
    
        action[12:24] = eef_to_action_single_arm(
            eef_pos=np.stack([PoseUtils.unmake_pose(seg1_arm2[i])[:3, 3]]),
            eef_rot_mats=np.stack([PoseUtils.unmake_pose(seg1_arm2[i])[:3, :3]]),
            original_action=seg1_arm2[i], # action_trajectory[0],
            prev_eef_pos=np.stack([obs['robot0_eef_pos'], obs['robot1_eef_pos']]),
            prev_eef_quat=np.stack([obs['robot0_eef_quat'], obs['robot1_eef_quat']]),
            max_dpos=max_dpos,
            max_drot=max_drot,
            cur_rot_from_env=False, #cTrue
        )

    else:   
        action[0:12] = eef_to_action_single_arm(
            eef_pos=np.stack([PoseUtils.unmake_pose(seg1_arm1[i])[:3, 3]]),
            eef_rot_mats=np.stack([PoseUtils.unmake_pose(seg1_arm1[i])[:3, :3]]),
            original_action=seg1_arm1[i], # action_trajectory[0],
            prev_eef_pos=np.stack([obs['robot0_eef_pos'], obs['robot1_eef_pos']]),
            prev_eef_quat=np.stack([obs['robot0_eef_quat'], obs['robot1_eef_quat']]),
            max_dpos=max_dpos,
            max_drot=max_drot,
            cur_rot_from_env=True
        )   
    
        action[12:24] = eef_to_action_single_arm(
            eef_pos=np.stack([PoseUtils.unmake_pose(seg1_arm2[i])[:3, 3]]),
            eef_rot_mats=np.stack([PoseUtils.unmake_pose(seg1_arm2[i])[:3, :3]]),
            original_action=seg1_arm2[i], # action_trajectory[0],
            prev_eef_pos=np.stack([obs['robot0_eef_pos'], obs['robot1_eef_pos']]),
            prev_eef_quat=np.stack([obs['robot0_eef_quat'], obs['robot1_eef_quat']]),
            max_dpos=max_dpos,
            max_drot=max_drot,
            cur_rot_from_env=True
        )

    prev_action = action.copy()

    obs, reward, done, info = env.step(action)
    env.render()
    time.sleep(0.05)


In [ ]:
'''
Execute interpolation to initial poses of segment 2 for each arm in parallel
'''

# obs = obfull[0]
# reset_to(env, initial_state) # Reset this to some other state
# reset_to_state(env, states_trajectory[0])
obfull = just_reset(env, initial_state) #  --- IGNORE ---

obs = obfull[0].copy()

N, M = len(interp_arm1_toseg2), len(interp_arm2_toseg2)

prev_action = action_trajectory[0]

for i in range(max(N, M)):

    action = prev_action.copy()

    if i == 0:
        if i < N:
            action[0:12] = eef_to_action_single_arm(
                eef_pos=np.stack([PoseUtils.unmake_pose(interp_arm1_toseg2[i])[:3, 3]]),
                eef_rot_mats=np.stack([PoseUtils.unmake_pose(interp_arm1_toseg2[i])[:3, :3]]),
                original_action=interp_arm1_toseg2[i], # action_trajectory[0],
                prev_eef_pos=np.stack([obs['robot0_eef_pos'], obs['robot1_eef_pos']]),
                prev_eef_quat=np.stack([obs['robot0_eef_quat'], obs['robot1_eef_quat']]),
                max_dpos=max_dpos,
                max_drot=max_drot,
                cur_rot_from_env=False, #cTrue
            )
        if i < M:
            action[12:24] = eef_to_action_single_arm(
                eef_pos=np.stack([PoseUtils.unmake_pose(interp_arm2_toseg2[i])[:3, 3]]),
                eef_rot_mats=np.stack([PoseUtils.unmake_pose(interp_arm2_toseg2[i])[:3, :3]]),
                original_action=interp_arm2_toseg2[i], # action_trajectory[0],
                prev_eef_pos=np.stack([obs['robot0_eef_pos'], obs['robot1_eef_pos']]),
                prev_eef_quat=np.stack([obs['robot0_eef_quat'], obs['robot1_eef_quat']]),
                max_dpos=max_dpos,
                max_drot=max_drot,
                cur_rot_from_env=False, #cTrue
            )

    else:   
        if i < N:
            action[0:12] = eef_to_action_single_arm(
                eef_pos=np.stack([PoseUtils.unmake_pose(interp_arm1_toseg2[i])[:3, 3]]),
                eef_rot_mats=np.stack([PoseUtils.unmake_pose(interp_arm1_toseg2[i])[:3, :3]]),
                original_action=interp_arm1_toseg2[i], # action_trajectory[0],
                prev_eef_pos=np.stack([obs['robot0_eef_pos'], obs['robot1_eef_pos']]),
                prev_eef_quat=np.stack([obs['robot0_eef_quat'], obs['robot1_eef_quat']]),
                max_dpos=max_dpos,
                max_drot=max_drot,
                cur_rot_from_env=True
            )   
        if i < M:
            action[12:24] = eef_to_action_single_arm(
                eef_pos=np.stack([PoseUtils.unmake_pose(interp_arm2_toseg2[i])[:3, 3]]),
                eef_rot_mats=np.stack([PoseUtils.unmake_pose(interp_arm2_toseg2[i])[:3, :3]]),
                original_action=interp_arm2_toseg2[i], # action_trajectory[0],
                prev_eef_pos=np.stack([obs['robot0_eef_pos'], obs['robot1_eef_pos']]),
                prev_eef_quat=np.stack([obs['robot0_eef_quat'], obs['robot1_eef_quat']]),
                max_dpos=max_dpos,
                max_drot=max_drot,
                cur_rot_from_env=True
            )

    prev_action = action.copy()

    obs, reward, done, info = env.step(action)
    env.render()
    time.sleep(0.05)


In [ ]:
'''
Execute transformed synced trajectory for each arm in parallel - segment 2

obs carried forward from previous loop
'''

# Assert both segments are of same length
assert len(seg2_arm1) == len(seg2_arm2)

N = len(seg2_arm1)

prev_action = action_trajectory[0]

for i in range(N):
    action = prev_action.copy()

    if i == 0:
        action[0:12] = eef_to_action_single_arm(
            eef_pos=np.stack([PoseUtils.unmake_pose(seg2_arm1[i])[:3, 3]]),
            eef_rot_mats=np.stack([PoseUtils.unmake_pose(seg2_arm1[i])[:3, :3]]),
            original_action=seg2_arm1[i], # action_trajectory[0],
            prev_eef_pos=np.stack([obs['robot0_eef_pos'], obs['robot1_eef_pos']]),
            prev_eef_quat=np.stack([obs['robot0_eef_quat'], obs['robot1_eef_quat']]),
            max_dpos=max_dpos,
            max_drot=max_drot,
            cur_rot_from_env=False, #cTrue
        )
    
        action[12:24] = eef_to_action_single_arm(
            eef_pos=np.stack([PoseUtils.unmake_pose(seg2_arm2[i])[:3, 3]]),
            eef_rot_mats=np.stack([PoseUtils.unmake_pose(seg2_arm2[i])[:3, :3]]),
            original_action=seg2_arm2[i], # action_trajectory[0],
            prev_eef_pos=np.stack([obs['robot0_eef_pos'], obs['robot1_eef_pos']]),
            prev_eef_quat=np.stack([obs['robot0_eef_quat'], obs['robot1_eef_quat']]),
            max_dpos=max_dpos,
            max_drot=max_drot,
            cur_rot_from_env=False, #cTrue
        )

    else:   
        action[0:12] = eef_to_action_single_arm(
            eef_pos=np.stack([PoseUtils.unmake_pose(seg2_arm1[i])[:3, 3]]),
            eef_rot_mats=np.stack([PoseUtils.unmake_pose(seg2_arm1[i])[:3, :3]]),
            original_action=seg2_arm1[i], # action_trajectory[0],
            prev_eef_pos=np.stack([obs['robot0_eef_pos'], obs['robot1_eef_pos']]),
            prev_eef_quat=np.stack([obs['robot0_eef_quat'], obs['robot1_eef_quat']]),
            max_dpos=max_dpos,
            max_drot=max_drot,
            cur_rot_from_env=True
        )   
    
        action[12:24] = eef_to_action_single_arm(
            eef_pos=np.stack([PoseUtils.unmake_pose(seg2_arm2[i])[:3, 3]]),
            eef_rot_mats=np.stack([PoseUtils.unmake_pose(seg2_arm2[i])[:3, :3]]),
            original_action=seg2_arm2[i], # action_trajectory[0],
            prev_eef_pos=np.stack([obs['robot0_eef_pos'], obs['robot1_eef_pos']]),
            prev_eef_quat=np.stack([obs['robot0_eef_quat'], obs['robot1_eef_quat']]),
            max_dpos=max_dpos,
            max_drot=max_drot,
            cur_rot_from_env=True
        )

    prev_action = action.copy()

    obs, reward, done, info = env.step(action)
    env.render()
    time.sleep(0.05)
